<a href="https://colab.research.google.com/github/heeboventure/Python/blob/main/Web_Scraping_Articles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기사 웹 스크래핑 & 첨부파일과 함께 다수에게 이메일 보내기

In [ ]:
# 기사 스크래핑 시작 코드

from bs4 import BeautifulSoup
from selenium import webdriver

driver = webdriver.Chrome('chromedriver')

url = "https://search.naver.com/search.naver?where=news&sm=tab_jum&query=추석"

driver.get(url)
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

### 아래 코드 참고 ###

driver.quit()

### .text 로 텍스트만 가져오기

In [ ]:
articles = soup.select_one('#sp_nws1 > dl > dt > a')

print(articles.text)

# 이렇게 .text 해주면 태그 중에서 텍스트만 가져온다.

### 가져오고자 하는 여러 개의 기사들 selector에 일정한 규칙이 없을 때 (이미지 웹 스크래핑과 달리)

- selector 코드를 보니 기사가 li로 되어있고 기사들을 모두 묶는 게 ul로 되어 있음 
- --> ul을 copy selector 해서 그 안의 li를 가져오는 방식 선택

In [ ]:
articles = soup.select('#main_pack > div.news.mynews.section._prs_nws > ul > li')

# 기사 제목과 보이는 내용 스크래핑
for article in articles:
    title = article.select_one('dl > dt > a').text
    content = article.select_one('dl > dd:nth-child(3)').text

### 기사 URL과 신문사 스크래핑

In [ ]:
# 기사 url과 언론사 이름도 소스코드 보면서 찾기
url = article.select_one('dl > dt > a')['href']
comp = article.select_one('span._sp_each_source').text.split('언론사')[0]

### 엑셀로 저장하기

- openpyxl 패키지 설치

In [ ]:
# 엑셀 저장 기본 코드
from openpyxl import Workbook

wb = Workbook()
ws1 = wb.active
ws1.title = "articles" #시트 이름 설정
ws1.append(["제목", "링크", "신문사"])

wb.save(filename='articles.xlsx') # 엑셀 저장

### 전체 코드 - 여러 기사 엑셀 저장

In [ ]:
from openpyxl import Workbook
from bs4 import BeautifulSoup
from selenium import webdriver

driver = webdriver.Chrome('chromedriver')

url = "https://search.naver.com/search.naver?where=news&sm=tab_jum&query=추석"

driver.get(url)
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

articles = soup.select('#main_pack > div.news.mynews.section._prs_nws > ul > li')

wb = Workbook()
ws1 = wb.active
ws1.title = "articles"
ws1.append(["제목", "링크", "신문사"])

for article in articles:
    title = article.select_one('dl > dt > a').text
    url = article.select_one('dl > dt > a')['href']
    comp = article.select_one('span._sp_each_source').text.split('언론사')[0]
    ws1.append([title, url, comp])

driver.quit()
wb.save(filename='articles.xlsx')

### 여러 사람에게 이메일 보내기

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders

# 보내는 사람 정보
me = "이메일@gmail.com"
my_password = "비밀번호"

# 로그인하기
s = smtplib.SMTP_SSL('smtp.gmail.com')
s.login(me, my_password)

# 받는 사람 정보 - 받을 여러 명의 주소를 List로!
emails = ['이메일1@naver.com', '이메일2@gmail.com']

for you in emails:
  # 메일 기본 정보 설정
  msg = MIMEMultipart('alternative')
  msg['Subject'] = "dear 여러 명"
  msg['From'] = me
  msg['To'] = you

  # 메일 내용 쓰기
  content = "여러 명에게 보내는 이메일"
  part2 = MIMEText(content, 'plain')
  msg.attach(part2)

  # 메일 보내고 서버 끄기
  s.sendmail(me, you, msg.as_string())

s.quit()

### 파일 첨부하기

In [ ]:
# 파일 첨부 코드
part = MIMEBase('application', "octet-stream")
with open("articles.xlsx", 'rb') as file:
    part.set_payload(file.read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment", filename="추석기사.xlsx") # '추석기사'라는 파일명으로 전송
msg.attach(part)

### 전체 코드 - 기사 스크래핑 및 다수에 첨부파일 이메일 전송

In [ ]:
import dload
from bs4 import BeautifulSoup
from selenium import webdriver
from openpyxl import Workbook
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders

driver = webdriver.Chrome('chromedriver')
url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'

driver.get(url)
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

wb = Workbook()
ws1 = wb.active
ws1.title = '인공지능 기사 모음'
ws1.append(['기사 제목', '링크', '신문사', '썸네일'])

## 기사 스크래핑
articles = soup.select('#main_pack > div.news.mynews.section._prs_nws > ul > li')

i = 1
for article in articles:
    #기사제목 스크래핑
    title = article.select_one('dl > dt > a').text
    #기사 url 스크래핑
    url = article.select_one('div > a')['href']
    #신문사 스크래핑
    comp = article.select_one('span._sp_each_source').text.split('언론사')[0]
    # 썸네일 스크래핑 & 이미지 저장
    thumb = article.select_one('div > a > img')['src']
    dload.save(thumb, f'article_thumbnail/{i}.jpg')
    i += 1

    ws1.append([title, url, comp, thumb])

driver.quit()
wb.save(filename='articles.xlsx')

## 여러 명에게 이메일 보내기

# 보내는 사람 정보
me = "이메일@gmail.com"
my_password = "비밀번호"

# 로그인하기
s = smtplib.SMTP_SSL('smtp.gmail.com')
s.login(me, my_password)

# 받는 사람 정보
yous = ['이메일1@naver.com', '이메일2@gmail.com']

for you in yous:
    # 메일 기본 정보 설정
    msg = MIMEMultipart('alternative')
    msg['Subject'] = "인공지능 기사 모음 파일 전달"
    msg['From'] = me
    msg['To'] = you

    # 메일 내용 쓰기
    content = "인공지능 관련 기사 모음 이메일 보내드립니다."
    part2 = MIMEText(content, 'plain')
    msg.attach(part2)

    # 파일 첨부하기
    part = MIMEBase('application', "octet-stream")
    with open("articles.xlsx", 'rb') as file:
        part.set_payload(file.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', "attachment", filename="AI기사.xlsx")
    msg.attach(part)

    # 메일 보내고 서버 끄기
    s.sendmail(me, you, msg.as_string())

s.quit()